In [3]:
import os
import glob
import numpy as np
import random
from scipy.signal import butter, sosfilt, resample_poly, stft
import soundfile as sf
from tqdm import tqdm
import librosa
# ————————————
# Your generalized processing function:
def process_audio_general(
    path,
    target_rates=[1000, 2000, 4000, 8000, 16000, 22000],
    high_freq_threshold=11000,
    significance_threshold=1e-5
):
    data, fs = sf.read(path)
    if data.ndim > 1:
        data = data[:, 0]
    chunk_size = fs
    n_chunks = len(data) // chunk_size
    raw_resampled   = {sr: [] for sr in target_rates}
    filtered_resampled = {sr: [] for sr in target_rates}
    power = []
    rat = []
    for i in range(n_chunks):
        chunk = data[i*chunk_size:(i+1)*chunk_size]
        # power spectral density
        psd = np.abs(np.fft.rfft(chunk))**2
        freqs = np.fft.rfftfreq(len(chunk), d=1/fs)
        total_power = psd.sum()
        mean_square_power = psd.sum() / (len(chunk)**2)
        # print("MSP", mean_square_power)
        if mean_square_power <= significance_threshold:
            continue
        high_power  = psd[freqs > high_freq_threshold].sum() / (len(chunk)**2)
        if high_power / mean_square_power < significance_threshold:
            continue
        # print("HTR", high_power / mean_square_power)
        for sr in target_rates:
            # raw resample → “unfiltered”
            raw = resample_poly(chunk, sr, fs)
            raw_resampled[sr].append(raw)

            # filter @ Nyquist(sr), then resample → “filtered”
            cutoff = sr / 2
            sos = butter(6, cutoff, fs=fs, btype='low', output='sos')
            filtered = sosfilt(sos, chunk)
            filt_rs = resample_poly(filtered, sr, fs)
            filtered_resampled[sr].append(filt_rs)

    return raw_resampled, filtered_resampled, fs

# ————————————
# STFT helper (fixed n_fft & 50% overlap):
def generate_stft(chunk, fs, n_fft=512):
    f, t, Zxx = stft(chunk, fs=fs, nperseg=n_fft, noverlap=n_fft//2)
    return np.abs(Zxx)

# ————————————
# 1) Gather all files + class IDs
def collect_file_list():
    files = [[], [], []]
    # UrbanSound8K
    base1 = r"D:\Aliasing3\UrbanSound8K\audio"
    for fold in tqdm(range(1, 11), desc="u8k"):
        for path in glob.glob(os.path.join(base1, f"fold{fold}", "*.wav")):
            cls = os.path.basename(path).split('-')[1]  # e.g. “7061-6-0-0.wav” → class “6”
            files[0].append({'path': path, 'class': cls})
    # ESC‑50
    base2 = r"D:\Aliasing3\ESC-50-master\audio"
    for path in tqdm(glob.glob(os.path.join(base2, "*.wav")), desc="esc"):
        name = os.path.basename(path).rsplit('.', 1)[0].split('-')
        cls = name[-1]  # “1-137-A-32.wav” → class “32”
        files[1].append({'path': path, 'class': cls})
    # Zenodo
    base3 = r"D:\Aliasing3\Zenodo\audio_train"
    for path in tqdm(glob.glob(os.path.join(base3, "*.flac")), desc="zen"):
        files[2].append({'path': path, 'class': '0'})  # no class → “0”
    return [files[0], files[1], files[2]]

# ————————————
# 2) First pass → global min/max of every STFT bin
def find_global_bounds(file_list):
    lo, hi = np.inf, -np.inf
    for entry in tqdm(file_list, desc="Global Bounds"):
        raw, filt, fs = process_audio_general(entry['path'])
        for d in (raw, filt):
            for sr, chunks in d.items():
                for chunk in chunks:
                    mag = generate_stft(chunk, fs)
                    lo = min(lo, mag.min())
                    hi = max(hi, mag.max())
    return lo, hi

# ————————————
# 3) Second pass → normalize & save .npy, then split
def build_and_save_dataset(file_list, lo, hi, out_root,
                           train_frac=0.8, val_frac=0.1, seed=42):
    os.makedirs(out_root, exist_ok=True)
    all_samples = []

    # --- collect with 'sr' in metadata ---
    for entry in tqdm(file_list, desc="Processing Samples"):
        raw, filt, fs = process_audio_general(entry['path'])
        for label, d in [('unfiltered', raw), ('filtered', filt)]:
            for sr, chunks in d.items():
                for chunk in chunks:
                    mag = generate_stft(chunk, sr)   # use sr as fs here!
                    norm = (mag - lo) / (hi - lo)
                    all_samples.append({
                        'array': norm.astype(np.float32),
                        'label': label,
                        'class': entry['class'],
                        'sr': sr
                    })

    # shuffle & split
    random.seed(seed)
    random.shuffle(all_samples)
    N = len(all_samples)
    n_train = int(train_frac * N)
    n_val   = int(val_frac  * N)

    splits = {
        'train':      all_samples[            :n_train],
        'validation': all_samples[n_train    :n_train+n_val],
        'test':       all_samples[n_train+n_val:]
    }

    # save out
    counters = {}
    for sp, samples in splits.items():
        for s in tqdm(samples, desc="Saving Files"):
            sr = s['sr']
            lbl = s['label']
            # initialize counter for this combination
            counters.setdefault((sr, sp, lbl), 0)
            counters[(sr, sp, lbl)] += 1

            idx = counters[(sr, sp, lbl)]
            fname = f"{idx}-{s['class']}.npy"

            # new path: out_root/{sr}/{split}/{label}
            save_dir = os.path.join(out_root, str(sr), sp, lbl)
            os.makedirs(save_dir, exist_ok=True)

            full_path = os.path.join(save_dir, fname)
            np.save(full_path, s['array'])


def create_mfcc_dataset(
    file_list,
    out_root,
    target_rates=[1000, 2000, 4000, 8000, 16000, 22000],
    n_mfcc=20,
    train_frac=0.8,
    val_frac=0.1,
    seed=42,
    high_freq_threshold=11000,
    significance_threshold=1e-5
):
    """
    Build a MFCC-based dataset from processed chunks, mirroring the STFT pipeline:
      - Uses process_audio_general to generate 'unfiltered' & 'filtered' chunks at each sr.
      - Computes MFCCs (n_mfcc coefficients) on each chunk (resampled at its sr).
      - Flattens each MFCC matrix to a 1D feature vector.
      - Shuffles and splits into train/validation/test (80/10/10 by default).
      - Normalizes features with z-score (mean/std) computed on TRAIN only.
      - Saves each flattened, normalized feature vector as .npy under:

        out_root/{sr}/{split}/{filtered|unfiltered}/<idx>-<class>.npy

      - Also saves normalization stats to out_root/mfcc_norm_stats.npz
    """
    os.makedirs(out_root, exist_ok=True)
    samples = []

    # 1) Gather all MFCC features
    for entry in tqdm(file_list, desc="Generating MFCC features"):
        raw_dict, filt_dict, _ = process_audio_general(
            entry['path'],
            target_rates=target_rates,
            high_freq_threshold=high_freq_threshold,
            significance_threshold=significance_threshold
        )
        for label, d in [('unfiltered', raw_dict), ('filtered', filt_dict)]:
            for sr, chunks in d.items():
                for chunk in chunks:
                    # compute MFCC on resampled chunk
                    mfcc = librosa.feature.mfcc(
                        y=chunk,
                        sr=sr,
                        n_mfcc=n_mfcc,
                        n_fft=sr//2,
                        hop_length=sr//4
                    )
                    feat = mfcc.flatten()
                    samples.append({
                        'feat': feat.astype(np.float32),
                        'label': label,
                        'class': entry['class'],
                        'sr': sr
                    })

    # 2) Shuffle & split
    random.seed(seed)
    random.shuffle(samples)
    N = len(samples)
    n_train = int(train_frac * N)
    n_val   = int(val_frac   * N)

    splits = {
        'train':      samples[:n_train],
        'validation': samples[n_train:n_train+n_val],
        'test':       samples[n_train+n_val:]
    }

    # 3) Compute normalization on TRAIN only
    feats_train = np.vstack([s['feat'] for s in splits['train']])
    mean = feats_train.mean(axis=0)
    std  = feats_train.std(axis=0) + 1e-8
    np.savez(os.path.join(out_root, 'mfcc_norm_stats.npz'), mean=mean, std=std)

    # 4) Save normalized features per split/sr/label
    counters = {}
    for split_name, split_samples in splits.items():
        for s in tqdm(split_samples, desc=f"Saving {split_name}"):
            sr    = s['sr']
            label = s['label']
            cls   = s['class']
            key = (sr, split_name, label)
            counters.setdefault(key, 0)
            counters[key] += 1
            idx = counters[key]

            # normalize
            norm_feat = (s['feat'] - mean) / std

            # path: out_root/sr/split/label
            save_dir = os.path.join(out_root, str(sr), split_name, label)
            os.makedirs(save_dir, exist_ok=True)
            fname = f"{idx}-{cls}.npy"
            np.save(os.path.join(save_dir, fname), norm_feat)

    print(f"MFCC dataset saved under {out_root}")
# ————————————
if __name__ == "__main__":
    file_list = collect_file_list()
    # lo1, hi1     = find_global_bounds(file_list[0])
    # lo2, hi2     = find_global_bounds(file_list[1])
    lo3, hi3     = find_global_bounds(file_list[2])
    # build_and_save_dataset(
    #     file_list[0],
    #     lo1, hi1,
    #     out_root=r"D:\Aliasing3\Processed_Files\DS_U8K"
    # )
    # build_and_save_dataset(
    #     file_list[1],
    #     lo2, hi2,
    #     out_root=r"D:\Aliasing3\Processed_Files\DS_ESC"
    # )
    build_and_save_dataset(
        file_list[2],
        lo3, hi3,
        out_root=r"D:\Aliasing3\Processed_Files\DS_ZEN"
    )
    # lists = collect_file_list()
    # out_roots = [
    #     # r"D:\Aliasing3\Processed_Files\DS_U8K_MFCC",
    #     # r"D:\Aliasing3\Processed_Files\DS_ESC_MFCC",
    #     r"D:\Aliasing3\Processed_Files\DS_ZEN_MFCC",
    # ]
    # for flist, root in zip(lists, out_roots):
    #     create_mfcc_dataset(flist, root)
    # print("Done! 🎉  Your train/validation/test splits are in Processed_Files.")

# **Key points:**
# - **Two‑pass normalization** ensures every `.npy` STFT uses the same min/max scale.
# - **80/10/10 split**, shuffled with a fixed seed for reproducibility.

Saving Files: 100%|███████████████████████████████████████████████████████████████| 6674/6674 [00:18<00:00, 351.77it/s]


In [4]:
import os
import glob
import numpy as np
import random
import pickle
import csv
from datetime import datetime
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

# Configuration
SAMPLE_RATES = [1000, 2000, 4000, 8000, 16000, 22000]

# Dataset roots for STFT and MFCC pipelines
DATASETS = {
    'STFT': [
        # r"D:\Aliasing3\Processed_Files\DS_ESC",
        r"D:\Aliasing3\Processed_Files\DS_U8K",
        r"D:\Aliasing3\Processed_Files\DS_ZEN"
    ],
    # 'MFCC': [
    #     r"D:\Aliasing3\Processed_Files\DS_ESC_MFCC",
    #     r"D:\Aliasing3\Processed_Files\DS_U8K_MFCC",
    #     r"D:\Aliasing3\Processed_Files\DS_ZEN_MFCC"
    # ]
}

# Collect results for CSV output
results = []

def load_data(root_dir, sr, split):
    """
    Load flattened feature .npy files for a given sample rate and split.
    Handles both STFT and MFCC datasets (features already flattened).
    Assumes structure: root_dir/{sr}/{split}/{filtered|unfiltered}/*.npy
    Returns X (n_samples, n_features), y (n_samples,)
    """
    X, y = [], []
    for label, cls in [('unfiltered', 0), ('filtered', 1)]:
        folder = os.path.join(root_dir, str(sr), split, label)
        if not os.path.isdir(folder):
            continue
        files = glob.glob(os.path.join(folder, '*.npy'))
        for fpath in tqdm(files, desc=f"Loading {split} {label} {sr}Hz"):
            arr = np.load(fpath)
            vec = arr.flatten()
            X.append(vec)
            y.append(cls)
    if not X:
        return np.empty((0, 0)), np.empty((0,))
    return np.vstack(X), np.array(y)


def balance_data(X, y, seed=42):
    """Down-sample majority class to match minority for balanced classes."""
    np.random.seed(seed)
    classes, counts = np.unique(y, return_counts=True)
    min_count = counts.min()
    idxs = []
    for cls in classes:
        cls_idxs = np.where(y == cls)[0]
        selected = np.random.choice(cls_idxs, min_count, replace=False)
        idxs.extend(selected)
    random.shuffle(idxs)
    return X[idxs], y[idxs]


def train_and_validate_for_rate(root_dir, sr, feature_type):
    print(f"\n=== {feature_type} | Root: {root_dir} | {sr}Hz ===")
    models_dir = os.path.join(root_dir, 'models')
    os.makedirs(models_dir, exist_ok=True)

    # Check for MFCC normalization stats
    norm_path = os.path.join(root_dir, 'mfcc_norm_stats.npz')
    have_norm = os.path.isfile(norm_path)
    if have_norm:
        stats = np.load(norm_path)
        mean_vec = stats['mean']
        std_vec  = stats['std']

    # Load and preprocess training data
    X_train, y_train = load_data(root_dir, sr, 'train')
    if X_train.size == 0:
        print(f"No training data at {sr}Hz in {root_dir}; skipping.")
        return
    if have_norm:
        X_train = (X_train - mean_vec) / std_vec
    X_train, y_train = balance_data(X_train, y_train)

    # Load and preprocess validation data
    X_val, y_val = load_data(root_dir, sr, 'validation')
    if X_val.size == 0:
        print(f"No validation data at {sr}Hz in {root_dir}; skipping.")
        return
    if have_norm:
        X_val = (X_val - mean_vec) / std_vec
    X_val, y_val = balance_data(X_val, y_val)

    # Base classifiers
    xgb_base = XGBClassifier(
        n_estimators=500,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    rf_base = AdaBoostClassifier(
        estimator=RandomForestClassifier(n_estimators=10, random_state=42),
        n_estimators=5,
        random_state=42
    )
    dt_base = AdaBoostClassifier(
        estimator=DecisionTreeClassifier(),
        n_estimators=5,
        random_state=42
    )

    # Grid search only for XGBoost
    print("Running grid search for XGBoost...")
    param_grid = {
        'max_depth': [5],
        'learning_rate': [0.01, 0.1]
    }
    grid = GridSearchCV(
        xgb_base,
        param_grid,
        cv=3,
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )
    grid.fit(X_train, y_train)
    best_xgb = grid.best_estimator_
    print("Best XGBoost params:", grid.best_params_)

    models = {
        'xgboost': best_xgb,
        'random_forest_boost': rf_base,
        'adaboost_dt': dt_base
    }

    # Train, evaluate, and save models
    for name, clf in models.items():
        print(f"Training {name}...")
        clf.fit(X_train, y_train)
        preds = clf.predict(X_val)
        acc = accuracy_score(y_val, preds)
        f1 = f1_score(y_val, preds, average='macro')
        print(f"Validation Accuracy ({name}): {acc:.4f}, F1 (macro): {f1:.4f}")
        print(classification_report(y_val, preds, target_names=['unfiltered', 'filtered']))

        # Save model
        model_path = os.path.join(models_dir, f"{name}_{sr}Hz.pkl")
        with open(model_path, 'wb') as f:
            pickle.dump(clf, f)
        print(f"Saved model to {model_path}")

        # Record results
        results.append({
            'feature_type': feature_type,
            'dataset_root': root_dir,
            'model': name,
            'sample_rate': sr,
            'f1_score': f1
        })


def main():
    for feature_type, roots in DATASETS.items():
        for root_dir in roots:
            for sr in SAMPLE_RATES:
                train_and_validate_for_rate(root_dir, sr, feature_type)

    # Save results to a new CSV with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_file = f"model_performance_{timestamp}.csv"
    with open(csv_file, 'w', newline='') as cf:
        writer = csv.DictWriter(cf, fieldnames=['feature_type', 'dataset_root', 'model', 'sample_rate', 'f1_score'])
        writer.writeheader()
        writer.writerows(results)
    print(f"Results saved to {csv_file}")
    print("All models trained and results recorded.")

if __name__ == '__main__':
    main()


=== STFT | Root: D:\Aliasing3\Processed_Files\DS_U8K | 1000Hz ===


Loading validation filtered 1000Hz: 100%|███████████████████████████████████████████| 618/618 [00:05<00:00, 103.04it/s]


Running grid search for XGBoost...
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best XGBoost params: {'learning_rate': 0.01, 'max_depth': 5}
Training xgboost...
Validation Accuracy (xgboost): 0.6737, F1 (macro): 0.6737
              precision    recall  f1-score   support

  unfiltered       0.67      0.67      0.67       596
    filtered       0.67      0.67      0.67       596

    accuracy                           0.67      1192
   macro avg       0.67      0.67      0.67      1192
weighted avg       0.67      0.67      0.67      1192

Saved model to D:\Aliasing3\Processed_Files\DS_U8K\models\xgboost_1000Hz.pkl
Training random_forest_boost...


C:\Users\jta20\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Validation Accuracy (random_forest_boost): 0.5285, F1 (macro): 0.5285
              precision    recall  f1-score   support

  unfiltered       0.53      0.53      0.53       596
    filtered       0.53      0.53      0.53       596

    accuracy                           0.53      1192
   macro avg       0.53      0.53      0.53      1192
weighted avg       0.53      0.53      0.53      1192

Saved model to D:\Aliasing3\Processed_Files\DS_U8K\models\random_forest_boost_1000Hz.pkl
Training adaboost_dt...


...

Running grid search for XGBoost...
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best XGBoost params: {'learning_rate': 0.1, 'max_depth': 5}
Training xgboost...
Validation Accuracy (xgboost): 0.7749, F1 (macro): 0.7748
              precision    recall  f1-score   support

  unfiltered       0.76      0.80      0.78       522
    filtered       0.79      0.75      0.77       522

    accuracy                           0.77      1044
   macro avg       0.78      0.77      0.77      1044
weighted avg       0.78      0.77      0.77      1044

Saved model to D:\Aliasing3\Processed_Files\DS_ZEN\models\xgboost_22000Hz.pkl
Training random_forest_boost...


C:\Users\jta20\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Validation Accuracy (random_forest_boost): 0.3190, F1 (macro): 0.3184
              precision    recall  f1-score   support

  unfiltered       0.33      0.35      0.34       522
    filtered       0.31      0.29      0.30       522

    accuracy                           0.32      1044
   macro avg       0.32      0.32      0.32      1044
weighted avg       0.32      0.32      0.32      1044

Saved model to D:\Aliasing3\Processed_Files\DS_ZEN\models\random_forest_boost_22000Hz.pkl
Training adaboost_dt...


C:\Users\jta20\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Validation Accuracy (adaboost_dt): 0.5412, F1 (macro): 0.5412
              precision    recall  f1-score   support

  unfiltered       0.54      0.54      0.54       522
    filtered       0.54      0.54      0.54       522

    accuracy                           0.54      1044
   macro avg       0.54      0.54      0.54      1044
weighted avg       0.54      0.54      0.54      1044

Saved model to D:\Aliasing3\Processed_Files\DS_ZEN\models\adaboost_dt_22000Hz.pkl
Results saved to model_performance_20250726_125159.csv
All models trained and results recorded.
